# SQL Analysis on Retail Sales Data

This notebook demonstrates how SQL can be used for business analysis on a real
retail dataset. SQLite is used within Python to simulate real-world SQL querying
and analytical workflows.

**Skills demonstrated:**
- SQL aggregations and filtering
- GROUP BY analysis
- Date-based analysis
- Customer and product-level insights


In [4]:
# ============================================================================
# 1. SQL ENVIRONMENT SETUP
# ============================================================================
print("="*60)
print("IMPORTING REQUIRED LIBRARIES")
print("="*60)

import pandas as pd
import sqlite3

print("Libraries imported successfully")



IMPORTING REQUIRED LIBRARIES
Libraries imported successfully


In [5]:
# ============================================================================
# 2. LOAD CLEANED RETAIL DATA INTO ENVIRONMENT
# ============================================================================

print("\n" + "=" * 60)
print("LOADING CLEANED RETAIL DATA")
print("=" * 60)

df = pd.read_csv("data/processed/cleaned_retail_sales.csv")

print("Dataset loaded successfully")
print(f"Shape of dataset: {df.shape}")

print("\nColumns:")
print(df.columns.tolist())




LOADING CLEANED RETAIL DATA
Dataset loaded successfully
Shape of dataset: (10000, 21)

Columns:
['Order_ID', 'Order_Date', 'Ship_Date', 'Customer_ID', 'Customer_Name', 'Segment', 'Region', 'Product_ID', 'Product_Category', 'Product_Sub_Category', 'Product_Name', 'Sales', 'Quantity', 'Discount', 'Profit', 'Shipping_Cost', 'Order_Priority', 'Unit_Price', 'Revenue', 'Sales_Original', 'Profit_Original']


In [6]:
# ============================================================================
# 3. BASIC DATA SANITY CHECKS
# ============================================================================

print("\n" + "=" * 60)
print("BASIC DATA SANITY CHECKS")
print("=" * 60)

print("Missing values:")
print(df.isnull().sum().sort_values(ascending=False).head())

print("\nDuplicate rows:", df.duplicated().sum())
print("Date range:", df["Order_Date"].min(), "to", df["Order_Date"].max())



STEP 3: BASIC DATA SANITY CHECKS
Missing values:
Order_ID         0
Order_Date       0
Ship_Date        0
Customer_ID      0
Customer_Name    0
dtype: int64

Duplicate rows: 0
Date range: 2022-01-01 00:00:00 to 2023-02-21 15:00:00


In [7]:
# ============================================================================
# 4. CREATE SQLITE DATABASE AND LOAD TABLE
# ============================================================================

print("\n" + "=" * 60)
print("CREATING SQLITE DATABASE")
print("=" * 60)

conn = sqlite3.connect("retail_analytics.db")

df.to_sql("retail_sales", conn, if_exists="replace", index=False)

print("Table 'retail_sales' created successfully")

# Verify table
tables = pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)
print("\nTables in database:")
print(tables)



STEP 4: CREATING SQLITE DATABASE
Table 'retail_sales' created successfully

Tables in database:
           name
0  retail_sales


In [8]:
# ============================================================================
# 5. BUSINESS SQL QUERIES
# ============================================================================

print("\n" + "=" * 60)
print("RUNNING BUSINESS SQL QUERIES")
print("=" * 60)

# 1. Total revenue by region
query_1 = """
SELECT Region, SUM(Sales) AS Total_Revenue
FROM retail_sales
GROUP BY Region
ORDER BY Total_Revenue DESC;
"""
print("\nRevenue by Region:")
print(pd.read_sql(query_1, conn))

# 2. Top 5 products by revenue
query_2 = """
SELECT Product_Name, SUM(Sales) AS Revenue
FROM retail_sales
GROUP BY Product_Name
ORDER BY Revenue DESC
LIMIT 5;
"""
print("\nTop 5 Products:")
print(pd.read_sql(query_2, conn))

# 3. Monthly sales trend
query_3 = """
SELECT strftime('%Y-%m', Order_Date) AS Month,
       SUM(Sales) AS Monthly_Revenue
FROM retail_sales
GROUP BY Month
ORDER BY Month;
"""
print("\nMonthly Revenue Trend:")
print(pd.read_sql(query_3, conn))





RUNNING BUSINESS SQL QUERIES

Revenue by Region:
    Region  Total_Revenue
0     East  318581.371468
1  Central  275688.179783
2     West  272330.955705
3    South  212070.472519

Top 5 Products:
  Product_Name      Revenue
0  Product_291  3970.096267
1    Product_2  3704.929597
2  Product_337  3663.076070
3   Product_69  3628.408737
4  Product_154  3592.281130

Monthly Revenue Trend:
      Month  Monthly_Revenue
0   2022-01     82670.481463
1   2022-02     72709.329106
2   2022-03     81062.839630
3   2022-04     75840.668735
4   2022-05     80056.231269
5   2022-06     76430.127987
6   2022-07     80699.415429
7   2022-08     82192.585990
8   2022-09     76398.577746
9   2022-10     77173.028698
10  2022-11     78702.980474
11  2022-12     82235.806623
12  2023-01     78408.906190
13  2023-02     54090.000133


In [12]:
# ----------------------------------------------------------------------------
# 6.Repeat vs One-Time Customers
# ----------------------------------------------------------------------------

query_repeat = """
SELECT
    CASE
        WHEN order_count > 1 THEN 'Repeat Customer'
        ELSE 'One-Time Customer'
    END AS Customer_Type,
    COUNT(*) AS Customers
FROM (
    SELECT Customer_ID, COUNT(DISTINCT Order_ID) AS order_count
    FROM retail_sales
    GROUP BY Customer_ID
)
GROUP BY Customer_Type;
"""
print("\n" + "=" * 60)
print("CUSTOMER TYPE DISTRIBUTION:")
print("=" * 60)
print(pd.read_sql(query_repeat, conn))



CUSTOMER TYPE DISTRIBUTION:
       Customer_Type  Customers
0  One-Time Customer         74
1    Repeat Customer       1912


In [13]:
# ----------------------------------------------------------------------------
# 7.Top 20% Customers Contribution
# ----------------------------------------------------------------------------

query_top_customers = """
SELECT
    Customer_ID,
    SUM(Sales) AS Total_Revenue
FROM retail_sales
GROUP BY Customer_ID
ORDER BY Total_Revenue DESC
LIMIT (
    SELECT CAST(COUNT(DISTINCT Customer_ID) * 0.2 AS INT)
    FROM retail_sales
);
"""
print("\n" + "=" * 60)
print("TOP REVENUE-CONTIBUTING CUSTOMERS:")
print("=" * 60)
print(pd.read_sql(query_top_customers, conn).head())



TOP REVENUE-CONTIBUTING CUSTOMERS:
  Customer_ID  Total_Revenue
0    CUST1097    1706.776827
1    CUST1800    1636.436219
2    CUST1939    1618.691144
3    CUST1463    1581.229018
4    CUST1295    1572.574497


In [14]:
# ----------------------------------------------------------------------------
# Average Delivery Time by Region
# ----------------------------------------------------------------------------

query_delivery = """
SELECT
    Region,
    AVG(
        JULIANDAY(Ship_Date) - JULIANDAY(Order_Date)
    ) AS Avg_Delivery_Days
FROM retail_sales
WHERE Ship_Date IS NOT NULL
GROUP BY Region
ORDER BY Avg_Delivery_Days;
"""
print("\n" + "=" * 60)
print("AVERAGE DELIVERY TIME BY REGION:")
print("=" * 60)
print(pd.read_sql(query_delivery, conn))



AVERAGE DELIVERY TIME BY REGION:
    Region  Avg_Delivery_Days
0  Central                1.0
1     East                1.0
2    South                1.0
3     West                1.0


In [20]:
# ============================================================================
# LOAD CUSTOMER CLV DATA
# ============================================================================

customer_clv = pd.read_csv("../data/processed/customer_clv.csv")

print("Customer CLV data loaded")
print("Shape:", customer_clv.shape)
print(customer_clv.head())



Customer CLV data loaded
Shape: (1986, 7)
  Customer_ID     Revenue  Orders                First                 Last  \
0    CUST0001  943.770842       8  2022-02-14 00:00:00  2023-01-14 04:00:00   
1    CUST0002  617.531725       6  2022-01-13 00:00:00  2023-01-28 14:00:00   
2    CUST0003  548.860692       5  2022-03-28 21:00:00  2023-01-15 05:00:00   
3    CUST0004  247.117878       3  2022-02-25 11:00:00  2022-11-26 17:00:00   
4    CUST0005  337.341185       4  2022-01-25 05:00:00  2022-11-04 19:00:00   

   Lifespan_Years          CLV  
0        0.915068  3094.099019  
1        1.041096  1779.466419  
2        0.800000  2058.227594  
3        0.750685   987.569620  
4        0.775342  1305.260061  


In [21]:
customer_clv.to_sql(
    "customer_clv",
    conn,
    if_exists="replace",
    index=False
)


1986

In [25]:
clv_40 = customer_clv["CLV"].quantile(0.4)
clv_80 = customer_clv["CLV"].quantile(0.8)

print("40th percentile CLV:", clv_40)
print("80th percentile CLV:", clv_80)


40th percentile CLV: 2049.01782618156
80th percentile CLV: 3504.4855468772384


In [26]:
query_clv_join = f"""
SELECT
    CASE
        WHEN c.CLV >= {clv_80} THEN 'High CLV'
        WHEN c.CLV >= {clv_40} THEN 'Medium CLV'
        ELSE 'Low CLV'
    END AS CLV_Segment,
    COUNT(DISTINCT r.Customer_ID) AS Customers,
    SUM(r.Sales) AS Total_Revenue,
    COUNT(DISTINCT r.Order_ID) AS Orders
FROM retail_sales r
JOIN customer_clv c
    ON r.Customer_ID = c.Customer_ID
GROUP BY CLV_Segment
ORDER BY Total_Revenue DESC;
"""

result = pd.read_sql(query_clv_join, conn)
print(result)


  CLV_Segment  Customers  Total_Revenue  Orders
0  Medium CLV        794  511376.650614    4647
1     Low CLV        794  294863.819532    3262
2    High CLV        398  272430.509330    2091


## Key SQL Insights

The SQL analysis highlights several important business patterns:

- Revenue is concentrated among a small subset of high-value customers
- Repeat customers contribute disproportionately to total sales
- Delivery performance varies by region, indicating potential operational bottlenecks
- Monthly sales trends confirm seasonality observed in Python-based analysis

This notebook demonstrates how SQL can be used alongside Python
to answer business questions efficiently at the database level.

